In [0]:
#csv desde drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
import numpy as np

Using TensorFlow backend.


## 3. *Encoder-Decoder* sobre Texto

Trabajos recientes en redes neuronales han demostrado que se puede aplicar a problemas bastante complejos gracias a la flexibilidad la definición de las redes, además de que se pueden adaptar a distintos tipos de datos brutos (dominios). Con el objetivo de explorar el enfoque anterior de *traducción* de algun tipo de dato, en esta sección deberá realizarlo con texto para traducción de un lenguaje humano a otro (e.g. inglés a alemán, chino a ruso).

In [0]:
df = pd.read_csv("drive/My Drive/Colab Notebooks/por.txt", sep="\t", names=["Source","Target"])

> a) Visualice los datos ¿Qué es la entrada y qué es la salida? Comente sobre los múltiples significados/sinónimos que puede tener una palabra al ser traducida y cómo propondría arreglar eso. *se espera que pueda implementarlo*

In [0]:
df.head(10)

,Source,Target
0,Go.,Vai.
1,Go.,Vá.
2,Hi.,Oi.
3,Run!,Corre!
4,Run!,Corra!
5,Run!,Corram!
6,Run.,Corre!
7,Run.,Corra!
8,Run.,Corram!
9,Who?,Quem?


La entrada del dataset corresponde a la frase o palabra escrita en ingles y el objetivo o salida corresponde a la traducción de lo anterior al idioma que se desea trabajar, en este caso portugues.

Al inicio del dataset se pueden ver unicamente palabras junto con su traducción explicita, se puede notar ademas que existen palabras repetidas en ingles que se traducen de una manera distinta, esto se debe a que el ingles no posee una conjugación explicita en la palabra como es el caso del español o el portuges, sino que se le da un contexto con el resto de la oración. Se espera que el encoder tenga problemas al traducir frases con dichas palabras.

In [0]:
df.tail()

,Source,Target
135666,"The Tatoeba Project, which can be found online...","O Projeto Tatoeba, que se pode encontrar on-li..."
135667,No matter how much you try to convince people ...,Não importa o quanto você tenta convencer os o...
135668,Some movies make such an impact that one never...,Alguns filmes são tão marcantes que jamais nos...
135669,A child who is a native speaker usually knows ...,Uma criança que é falante nativa geralmente sa...
135670,We recommend adding sentences and translations...,Recomendamos acrescentar frases e traduções na...


Cuando se analizan los datos al final del conjunto se puede ver que ya se esta trabajando con oraciones y no unicamente con palabras simples, se espera que estos ejemplos permitan a la red neuronal aprender a reconocer un poco el contexto en el cual se esta utilizando la palabra.

> b) Realice un pre-procesamiento a los textos como se acostumbra para eliminar símbolos inecesarios u otras cosas que estime conveniente, comente sobre la importancia de éste paso. Además de ésto deberá agregar un símbolo al final de la sentencia *target* para indicar un "alto" cuando la red neuronal necesite aprender a generar una sentencia.

In [0]:
table = str.maketrans('', '', string.punctuation)

def clean_text(text, where=None):
    """ OJO: Sin eliminar el significado de las palabras."""
    text = text.lower()
    tokenize_text = text.split()
    tokenize_text = [word.translate(table) for word in tokenize_text]#eliminar puntuacion
    tokenize_text = [word for word in tokenize_text if word.isalpha()] #remove numbers
    if where == 'target':
        tokenize_text = ["BOS"] + tokenize_text + ["EOS"] 
    return tokenize_text

  
texts_input = list(df['Source'].apply(clean_text))
texts_output = list(df['Target'].apply(clean_text, where='target'))

> Cree un conjunto de validación y de pruebas fijos de $N_{exp} = 10000$ datos ¿Cuántos datos quedan para entrenar? 

In [0]:
N_exp = 10000

In [0]:
X_train_l, X_test_l, Y_train_l, Y_test_l = train_test_split(texts_input, texts_output,
                                                            test_size=N_exp, random_state=22)
X_train_l, X_val_l, Y_train_l, Y_val_l = train_test_split(X_train_l, Y_train_l, 
                                                          test_size=N_exp, random_state=22)

In [0]:
len(X_train_l)

115671

Luego de haber generado el cconjunto de testing y validación aún quedan 115671 ejemplos para entrenar el modelo.

> *Recuerde que si no puede procesar los datos de entrenamiento adecuadamente siempre puede muestrear en base a la capacidad de cómputo que posea*

> c) Genere un vocabulario, **desde el conjunto de entrenamiento**, sobre las palabras a recibir y generar en la traducción, esto es codificarlas a un valor entero que servirá para que la red las vea en una representación útil a procesar, *comience desde el 1 debido a que el cero será utilizado más adelante*. Para reducir el vocabulario considere las palabras que aparecen un mínimo de *min_count* veces en todo los datos, se aconseja un valor de 3. Comente sobre la importancia de ésto al reducir el vocabulario ¿De qué tamaño es el vocabulario de entrada y salida? ¿La diferencia de ésto podría ser un factor importante?

In [0]:
def create_vocab(texts, min_count=1):
    count_vocab = {}
    for sentence in texts:
        for word in sentence:
            if word not in count_vocab:
                count_vocab[word] = 1
            else:
                count_vocab[word] += 1
    return [word for word,count in count_vocab.items() if count >= min_count]
  
vocab_source = create_vocab(X_train_l, min_count=3)
word2idx_s = {w: i+1 for i, w in enumerate(vocab_source)} #index (i+1) start from 1,2,3,...
idx2word_s = {i+1: w for i, w in enumerate(vocab_source)}
n_words_s = len(vocab_source)
vocab_target = create_vocab(Y_train_l, min_count=3)
word2idx_t = {w: i+1 for i, w in enumerate(vocab_target)}  #Converting text to numbers
idx2word_t = {i+1: w for i, w in enumerate(vocab_target)}
n_words_t = len(vocab_target)

Dado que existe una gran cantidad de palabras dentro del vocabulario y la memoria del sistema que se utilice no es ilimitada es necesario reducir el tamaño para poder trabajar con el dataset, ademas palabras que aparescan menos de una determinada cantidad de veces pueden considerarse como poco relevantes para la traducción del texto.

In [0]:
print("Tamaño de palabras en origen:",n_words_s)
print("\nTamaño de palabras en destino",n_words_t)

Tamaño de palabras en origen: 5954

Tamaño de palabras en destino 8761


El tamaño del conjunto de palabras de origen es de 5954 palabras, mientras que el destino es de 8760, esto se debe a lo comentado anteriormente referenciando a que el ingles obtiene la conjugación de una palabra en base al contexto en el cual se esta hablando y no explicitamente en la forma en que se escribe.

Esto, en opinión propia, puede afectar negativamente a la red, puesto que sin el contexto adecuado una red podría traducir una palabra de multiples maneras.

> Ahora codifique las palabras a los números indexados con el vocabulario. Recuerde que si una palabra en los otros conjuntos, o en el mismo de entrenamiento, no aparece en el vocabulario no se podrá generar una codificación, por lo que será **ignorada** ¿Cómo se podría evitar ésto?

In [0]:
""" Source/input data """
dataX_train = [[word2idx_s[word] for word in sent if word in word2idx_s] for sent in X_train_l]
dataX_val = [[word2idx_s[word] for word in sent if word in word2idx_s] for sent in X_val_l]
dataX_test = [[word2idx_s[word] for word in sent if word in word2idx_s] for sent in X_test_l]

""" Target/output data """
dataY_train = [[word2idx_t[word] for word in sent if word in word2idx_t] for sent in Y_train_l]
dataY_val = [[word2idx_t[word] for word in sent if word in word2idx_t] for sent in Y_val_l] 
dataY_test = [[word2idx_t[word] for word in sent if word in word2idx_t] for sent in Y_test_l] 

> d) Debido al largo variable de los textos de entrada y salida será necesario estandarizar ésto para poder trabajar de manera más cómoda en Keras, *cada texto (entrada y salida) pueden tener distinto largo máximo*. Comente sobre la decisión del tipo de *padding*, *pre o post* ¿Qué sucede al variar el largo máximo de instantes de tiempo para procesar en cada parte del modelo (entrada y salida)?

In [0]:
""" INPUT DATA (Origin language) """
max_inp_length = max([max(map(len,dataX_train)), max(map(len,dataX_val)), max(map(len,dataX_test))])
print("Largo max inp: ",max_inp_length)
word2idx_s["*"] = 0 #padding symbol
idx2word_s[0] = "*"
n_words_s += 1  
X_train = sequence.pad_sequences(dataX_train, maxlen=max_inp_length, padding='post', value=word2idx_s["*"])
X_val = sequence.pad_sequences(dataX_val, maxlen=max_inp_length, padding='post', value=word2idx_s["*"])
X_test = sequence.pad_sequences(dataX_test, maxlen=max_inp_length, padding='post', value=word2idx_s["*"])

""" OUTPUT DATA (Destination language) """
max_out_length = max([max(map(len,dataY_train)), max(map(len,dataY_val)), max(map(len,dataY_test))])
print("Largo max out: ",max_out_length)
word2idx_t["*"] = 0 #padding symbol
idx2word_t[0] = "*"
n_words_t += 1  
Y_train = sequence.pad_sequences(dataY_train, maxlen=max_out_length, padding='post', value=word2idx_t["*"])
Y_val = sequence.pad_sequences(dataY_val, maxlen=max_out_length, padding='post', value=word2idx_t["*"])
Y_test = sequence.pad_sequences(dataY_test, maxlen=max_out_length, padding='post', value=word2idx_t["*"])


Largo max inp:  35
Largo max out:  35


> e) Para evitar que la red obtenga una ganancia por imitar/predecir el símbolo de *padding* que está bastante presente en los datos coloque un peso sobre éste clase, con valor 0, así se evita que tenga impacto en la función objetivo. Ya que *keras* no soporta directamente ésto en series de tiempo coloque el peso a cada instante de tiempo de cada dato de entrenamiento dependiendo de su clase. Comente sobre alguna otra forma en que se podría manejar el evitar que la red prediga en mayoría el símbolo de *padding*.

In [0]:
c_weights = np.ones(n_words_t)
c_weights[0] = 0 #padding class masked
sample_weight = np.zeros(Y_train.shape)
for i in range(sample_weight.shape[0]):
    sample_weight[i] = c_weights[Y_train[i,:]]

> f) Para lograr la tarea defina una red recurrente del tipo *encoder*-*decoder* como la que se presenta en la siguiente imágen.
<img src="https://chunml.github.io/ChunML.github.io/images/projects/sequence-to-sequence/repeated_vector.png" width="60%" />
En primer lugar defina el *Encoder* que procesara el texto de entrada y retornará un solo vector final, haciendo uso de las capas ya conocidas de *Embedding* para generar un vector denso de palabra y *GRU*, pero en su versión acelerada para GPU.

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU, GRU
EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(input_dim=n_words_s, output_dim=EMBEDDING_DIM, input_length=max_inp_length))
model.add(CuDNNGRU(64, return_sequences=True))
model.add(CuDNNGRU(128, return_sequences=False))
#model.add(GRU(64, return_sequences=True))
#model.add(GRU(128, return_sequences=False))

W0626 17:46:41.052778 139758613624704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0626 17:46:41.088880 139758613624704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0626 17:46:41.096385 139758613624704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



> Luego defina la sección que conecta el largo (*timesteps*) de entrada *vs* el de salida.

In [0]:
from keras.layers import RepeatVector
model.add(RepeatVector(max_out_length)) #conection

Finalmente defina el *Decoder* para generar la secuencia de salida en texto de palabras en otro idioma, a través de la función *softmax* sobre cada instante de tiempo (*timestep*). 

In [0]:
from keras.layers import GRU, TimeDistributed,Dense
model.add(CuDNNGRU(128, return_sequences=True))
model.add(CuDNNGRU(64, return_sequences=True))
#model.add(GRU(64, return_sequences=True))
#model.add(GRU(128, return_sequences=True))
model.add(TimeDistributed(Dense(n_words_t, activation='softmax')))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 35, 100)           595500    
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (None, 35, 64)            31872     
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (None, 128)               74496     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 35, 128)           0         
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (None, 35, 128)           99072     
_________________________________________________________________
cu_dnngru_4 (CuDNNGRU)       (None, 35, 64)            37248     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 35, 8762)          569530    
Total para

> Entrene la red entre 1 a 5 *epochs*, agregando los pesos definidos sobre cada ejemplo de entrenamiento. Además de utilizar una función de pérdida que evita generar explícitamente los *one hot vector*


Antes de entrenar es necesario cambiar la forma del output a 3 dimensiones

In [0]:
Y_train = np.reshape(Y_train, (Y_train.shape[0], max_out_length, 1))
Y_val = np.reshape(Y_val, (Y_val.shape[0], max_out_length, 1))
Y_test = np.reshape(Y_test, (Y_test.shape[0], max_out_length, 1))

Y_train.shape
#X_test = np.reshape(X_test, (X_test.shape[0], lstm_num_timesteps, lstm_num_features))

(115671, 35, 1)

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', sample_weight_mode='temporal')
model.fit(X_train, Y_train, epochs=3, batch_size=256,validation_data=(X_val, Y_val), sample_weight = sample_weight)

W0626 17:46:43.106671 139758613624704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0626 17:46:43.137966 139758613624704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3341: The name tf.log is deprecated. Please use tf.math.log instead.

W0626 17:46:43.243191 139758613624704 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0626 17:46:43.954685 139758613624704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated

Train on 115671 samples, validate on 10000 samples
Epoch 1/3
115671/115671 [==============================] - 50s 434us/step - loss: 5.6570 - val_loss: 12.1736
Epoch 2/3
115671/115671 [==============================] - 43s 373us/step - loss: 5.3583 - val_loss: 12.8971
Epoch 3/3
115671/115671 [==============================] - 43s 375us/step - loss: 5.2091 - val_loss: 13.4141


> g) Debido a lo costoso de tener una red completamente recurrente para entrenar y poder experimentar, cambie el modelo que procesa el *Encoder* por una red convolucional, reduciendo el número de capas pero aumentando las neuronas. Utilice tamaños de *kernel*  igual a 5 y funciones de activaciones relu. Se agregan capas de *BatchNormalization* debido a que en el *Decoder* contamos con redes recurrentes que tienen capa activación distinta a la usada por las convoluciones. La capa de *GlobalMaxPooling1d* es lo que permite reducir toda la información extraída a un único vector, como se realizó anteriormente con *return_sequences=False*, comente sobre la ganancia o desventaja de ésto *vs* la red neuronal.

In [0]:
from keras.layers import Conv1D,MaxPool1D,GlobalMaxPooling1D,GlobalAveragePooling1D,BatchNormalization, InputLayer
modelc = Sequential()
modelc.add(Embedding(input_dim=n_words_s, output_dim=EMBEDDING_DIM, input_length=max_inp_length))
modelc.add(Conv1D(256, 5, padding='same', activation='relu', strides=1))
modelc.add(BatchNormalization()) #for stability
modelc.add(Conv1D(256, 5, padding='same', activation='relu', strides=1))
modelc.add(BatchNormalization())
modelc.add(GlobalMaxPooling1D()) #aka to return_sequences=False
modelc.add(RepeatVector(max_out_length)) #conection
modelc.add(CuDNNGRU(256, return_sequences=True))
modelc.add(TimeDistributed(Dense(n_words_t, activation='softmax')))
modelc.compile(optimizer='adam', loss='sparse_categorical_crossentropy', sample_weight_mode='temporal')
modelc.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 35, 100)           595500    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 35, 256)           128256    
_________________________________________________________________
batch_normalization_1 (Batch (None, 35, 256)           1024      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 35, 256)           327936    
_________________________________________________________________
batch_normalization_2 (Batch (None, 35, 256)           1024      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 35, 256)           0         
__________

> Entrene el modelo igual a lo presentado anteriormente pero ahora por 20 *epochs* ¿Cambian los tiempos de procesamiento y la cantidad de parámetros?

In [0]:
modelc.fit(X_train, Y_train, epochs=20, batch_size=256,validation_data=(X_val, Y_val), sample_weight = sample_weight)

Train on 115671 samples, validate on 10000 samples
Epoch 1/20
115671/115671 [==============================] - 58s 499us/step - loss: 4.8021 - val_loss: 13.2694
Epoch 2/20
115671/115671 [==============================] - 55s 480us/step - loss: 3.8034 - val_loss: 13.3340
Epoch 3/20
115671/115671 [==============================] - 56s 484us/step - loss: 3.1964 - val_loss: 13.2349
Epoch 4/20
115671/115671 [==============================] - 56s 486us/step - loss: 2.8064 - val_loss: 13.1902
Epoch 5/20
115671/115671 [==============================] - 56s 486us/step - loss: 2.5269 - val_loss: 13.1602
Epoch 6/20
115671/115671 [==============================] - 56s 486us/step - loss: 2.3129 - val_loss: 13.1418
Epoch 7/20
115671/115671 [==============================] - 56s 488us/step - loss: 2.1493 - val_loss: 13.0869
Epoch 8/20
115671/115671 [==============================] - 56s 487us/step - loss: 2.0186 - val_loss: 13.0770
Epoch 9/20
115671/115671 [==============================] - 56s 487us

> h) Visualice lo aprendido por el modelo sobre algunos datos del conjunto de entrenamiento y validación, comente lo observado.

In [0]:
def predict_words(y_indexs, data="target"):
    """ Predict until '-#anvorgesa-' is seen """
    return_val = []
    for indx_word in y_indexs:
        if indx_word != 0: #start to predict
            return_val.append(np.squeeze(indx_word))
            if data == "target": #if target is predicting
                if indx_word == word2idx_t["EOS"]:
                    return return_val                
    return return_val
  
n_s = 30
idx = np.random.choice(np.arange(Y_test.shape[0]), size=n_s, replace=False)

In [0]:
print("For the first model:")
Y_test_pred = model.predict_classes(X_test[idx])
for i, n_sampled in enumerate(idx):
    text_input = [idx2word_s[p] for p in predict_words(X_test[n_sampled], data="source")]
    print("\nText source: ", ' '.join(text_input))
    text_real = [idx2word_t[p] for p in predict_words(Y_test[n_sampled,:,0], data="target")]
    print("Text target true: ", ' '.join( text_real))
    text_sampled = [idx2word_t[p] for p in predict_words(Y_test_pred[i], data="target")]
    print("Text target predicted: ", ' '.join(text_sampled))

For the first model:

Text source:  will get lower
Text target true:  BOS as EOS
Text target predicted:  BOS EOS

Text source:  hes shorter than tom
Text target true:  BOS ele é mais baixo do que tom EOS
Text target predicted:  BOS EOS

Text source:  wait in here
Text target true:  BOS espere aqui EOS
Text target predicted:  BOS EOS

Text source:  youre bluffing
Text target true:  BOS vocês estão blefando EOS
Text target predicted:  BOS EOS

Text source:  they bought a car
Text target true:  BOS eles compraram um carro EOS
Text target predicted:  BOS EOS

Text source:  fog has limited to meters
Text target true:  BOS o a a metros EOS
Text target predicted:  BOS EOS

Text source:  she said that she was eager to go there
Text target true:  BOS ela disse que estava ansiosa para ir lá EOS
Text target predicted:  BOS EOS

Text source:  is the of life
Text target true:  BOS a variedade é o da vida EOS
Text target predicted:  BOS EOS

Text source:  we are looking forward to hearing about your

In [0]:
print("For the convolutional model:")
Y_test_pred = modelc.predict_classes(X_test[idx] )
for i, n_sampled in enumerate(idx):
    text_input = [idx2word_s[p] for p in predict_words(X_test[n_sampled], data="source")]
    print("\nText source: ", ' '.join(text_input))
    text_real = [idx2word_t[p] for p in predict_words(Y_test[n_sampled,:,0], data="target")]
    print("Text target true: ", ' '.join( text_real))
    text_sampled = [idx2word_t[p] for p in predict_words(Y_test_pred[i], data="target")]
    print("Text target predicted: ", ' '.join(text_sampled))

For the convolutional model:

Text source:  will get lower
Text target true:  BOS as EOS
Text target predicted:  BOS vai pegar EOS

Text source:  hes shorter than tom
Text target true:  BOS ele é mais baixo do que tom EOS
Text target predicted:  BOS ele é mais do do tom tom EOS

Text source:  wait in here
Text target true:  BOS espere aqui EOS
Text target predicted:  BOS espere aqui EOS

Text source:  youre bluffing
Text target true:  BOS vocês estão blefando EOS
Text target predicted:  BOS você está blefando EOS

Text source:  they bought a car
Text target true:  BOS eles compraram um carro EOS
Text target predicted:  BOS eles um carro carro EOS

Text source:  fog has limited to meters
Text target true:  BOS o a a metros EOS
Text target predicted:  BOS a metros metros metros EOS

Text source:  she said that she was eager to go there
Text target true:  BOS ela disse que estava ansiosa para ir lá EOS
Text target predicted:  BOS ela disse que que ir ir ir EOS

Text source:  is the of lif

> i) Realice algún cambio esperando que mejore el modelo entrenado, luego vuelva a visualizar lo predicho por la red *vs* lo real. *Debido a lo costoso en entrenar puede optar por realizar solo un cambio pero que sea significativo*.  Se comentan algunas opciones para utilizar y combinar:
* Cambiar  el *embedding* por alguno pre-entrenado
* Agregar regularizadores
* Asignar peso a las clases/palabras de salida
* Cambiar *Global max pooling* por *Average max pooling*
* Aumentar o reducir capas
* Aumentar o reducir neuronas/unidades  

In [0]:
modeli = Sequential()
modeli.add(Embedding(input_dim=n_words_s, output_dim=EMBEDDING_DIM, input_length=max_inp_length))
modeli.add(Conv1D(256, 5, padding='same', activation='relu', strides=1))
modeli.add(BatchNormalization()) #for stability
modeli.add(Conv1D(256, 5, padding='same', activation='relu', strides=1))
modeli.add(BatchNormalization())
modeli.add(Conv1D(256, 5, padding='same', activation='relu', strides=1))
modeli.add(BatchNormalization())
modeli.add(GlobalAveragePooling1D()) #aka to return_sequences=False
modeli.add(RepeatVector(max_out_length)) #conection
modeli.add(CuDNNGRU(512, return_sequences=True))
modeli.add(TimeDistributed(Dense(n_words_t, activation='softmax')))
modeli.compile(optimizer='adam', loss='sparse_categorical_crossentropy', sample_weight_mode='temporal')
modeli.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 35, 100)           595500    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 35, 256)           128256    
_________________________________________________________________
batch_normalization_6 (Batch (None, 35, 256)           1024      
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 35, 256)           327936    
_________________________________________________________________
batch_normalization_7 (Batch (None, 35, 256)           1024      
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 35, 256)           327936    
_________________________________________________________________
batch_normalization_8 (Batch (None, 35, 256)           1024      
__________

In [0]:
modeli.fit(X_train, Y_train, epochs=20, batch_size=256,validation_data=(X_val, Y_val), sample_weight = sample_weight)

Train on 115671 samples, validate on 10000 samples
Epoch 1/20
115671/115671 [==============================] - 76s 655us/step - loss: 4.5896 - val_loss: 13.6316
Epoch 2/20
115671/115671 [==============================] - 78s 671us/step - loss: 3.2544 - val_loss: 13.3680
Epoch 3/20
115671/115671 [==============================] - 79s 682us/step - loss: 2.5319 - val_loss: 13.5044
Epoch 4/20
115671/115671 [==============================] - 79s 684us/step - loss: 2.1046 - val_loss: 13.2607
Epoch 5/20
115671/115671 [==============================] - 79s 683us/step - loss: 1.8460 - val_loss: 13.2080
Epoch 6/20
115671/115671 [==============================] - 79s 683us/step - loss: 1.6674 - val_loss: 13.1604
Epoch 7/20
115671/115671 [==============================] - 79s 682us/step - loss: 1.5349 - val_loss: 13.1054
Epoch 8/20
115671/115671 [==============================] - 79s 682us/step - loss: 1.4275 - val_loss: 13.2484
Epoch 9/20
115671/115671 [==============================] - 79s 681us

In [0]:
print("For the convolutional model:")
Y_test_pred = modeli.predict_classes(X_test[idx] )
for i, n_sampled in enumerate(idx):
    text_input = [idx2word_s[p] for p in predict_words(X_test[n_sampled], data="source")]
    print("\nText source: ", ' '.join(text_input))
    text_real = [idx2word_t[p] for p in predict_words(Y_test[n_sampled,:,0], data="target")]
    print("Text target true: ", ' '.join( text_real))
    text_sampled = [idx2word_t[p] for p in predict_words(Y_test_pred[i], data="target")]
    print("Text target predicted: ", ' '.join(text_sampled))

For the convolutional model:

Text source:  will get lower
Text target true:  BOS as EOS
Text target predicted:  BOS você tentar acalmar EOS

Text source:  hes shorter than tom
Text target true:  BOS ele é mais baixo do que tom EOS
Text target predicted:  BOS ele é mais do do que também EOS

Text source:  wait in here
Text target true:  BOS espere aqui EOS
Text target predicted:  BOS espere aqui aqui EOS

Text source:  youre bluffing
Text target true:  BOS vocês estão blefando EOS
Text target predicted:  BOS você está blefando EOS

Text source:  they bought a car
Text target true:  BOS eles compraram um carro EOS
Text target predicted:  BOS eles compraram um um EOS

Text source:  fog has limited to meters
Text target true:  BOS o a a metros EOS
Text target predicted:  BOS a que tem de EOS

Text source:  she said that she was eager to go there
Text target true:  BOS ela disse que estava ansiosa para ir lá EOS
Text target predicted:  BOS ela disse que estava lá lá de lá lá EOS

Text sour

> j) A pesar de que la tarea de medir qué tan similar es un texto a otro ya es un área de investigación propia [[6]](#refs), usted deberá utilizar alguna métrica de desempeño para ver qué tan buena es la traducción del texto *versus* el texto real entregado. Debido a que la métrica de *Exact Matching* (EM) puede ser muy drástica, mida *f1 score* por texto además de proponer alguna otra técnica de evaluación para medir sobre el conjunto de pruebas y los otros conjuntos si estima conveniente. Puede basarse en otros trabajos como *Image captioning* o *Text summary*. 

> *Hint: Debido a los problemas de memoria al realizar un forward-pass, solo seleccione un subconjunto $N_{sub}$ del conjunto de pruebas para realizar ésta evaluación, se aconseja entre 1000 y 5000.*

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_score, recall_score

m = MultiLabelBinarizer().fit([np.arange(n_words_t)])

def calculate_f1(true, pred):
    true = np.squeeze(true)
    pred = np.squeeze(pred)
    binarized_true = m.transform([predict_words(true)])[0] #onehot of words appear
    binarized_pred = m.transform([predict_words(pred)])[0] #onehot of words appear
    return f1_score(binarized_true, binarized_pred, average='binary') #only on appearing words
  
ids = np.random.choice(np.arange(Y_test.shape[0]), size=3000, replace=False)

def evaluate_it_f1(ids, model):
  Y_true = [Y_test[i] for i in ids]
  x_t = np.asarray([X_test[i] for i in ids])

  Y_hat = modeli.predict_classes(x_t)

  f1_final = np.mean([calculate_f1(true_words,pred_words) for true_words,pred_words in zip(Y_true,Y_hat)])
  return f1_final #porcentaje

evaluate_it_f1(ids, modeli)


0.6829066388700148

>> La función de *f1 score* en este extracto se calcula en base al *precision* y *recall* de que aparezca cada una de las palabras predichas dentro de las palabras reales (como si cada palabra fuera una clase de "aparece" o no), **sin importar el orden ni la ocurrencia**.

> k) En ves de volver a variar el modelo de *Encoder*, dejaremos una representación manual explícita (*no entrenable*) a través de extraer características manuales de los textos *source*, como por ejemplo representaciones *term frequency* (TF) o TF-IDF, proporcionadas a través de __[sklearn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text)__. Luego, con esto generado, defina y entrene el modelo *Decoder* neuronal como el presentado en las preguntas anteriores, ésto es comenzar desde la capa *RepeatVector* hasta llegar a la clasificación sobre el texto *target*. Compare el desempeño con lo presentado en (j) y lo visualizado en (h).

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
def dummy_fun(doc):
    return doc
tf_idf = TfidfVectorizer(analyzer='word',tokenizer=dummy_fun,preprocessor=dummy_fun,
                         token_pattern=None,use_idf= True, smooth_idf=True, norm='l2')   
X_train_tfidf = tf_idf.fit_transform(dataX_train).astype('float32').todense()
X_val_tfidf = tf_idf.fit_transform(dataX_val).astype('float32').todense()
X_test_tfidf = tf_idf.fit_transform(dataX_test).astype('float32').todense()


In [0]:
X_train_tfidf.shape

(115671, 5954)

In [0]:
from keras.layers import InputLayer

modelf = Sequential()

modelf.add(RepeatVector(max_out_length)) #conection
modelf.add(CuDNNGRU(512, return_sequences=True))
modelf.add(TimeDistributed(Dense(n_words_t, activation='softmax')))
modelf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', sample_weight_mode='temporal')
modelf.summary() 

NameError: ignored